<a href="https://colab.research.google.com/github/Akonobea/bias-aware-explain-as-you-trade/blob/main/notebooks/01_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 16.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: pandas
    Found existing in

In [4]:
# ===============================================================
# 📊 01_data_collection.ipynb
# Bias-Aware Explain-as-you-Trade AI Agent
# Author: Josephine Akonobea Bekoe
# ===============================================================

# --- 1. Import Libraries ---
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np
import os
from datetime import datetime

In [8]:
TICKERS = ["AAPL", "MSFT", "JPM"]   # Start with 3 stocks (Tech + Finance mix)
start_date = "2020-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")


#Ensuring that directory exists

os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)

#Downloading the raw data from yf finance
for ticker in TICKERS:
    df = yf.download(ticker, start=start_date, end=end_date)
    df.reset_index(inplace=True)
    df.to_csv(f"data/raw/{ticker}.csv", index=False)
    all_data[ticker] = df
    print(f"✅ Downloaded {ticker} data: {df.shape[0]} rows")

# =====================================================
# 2️⃣ Feature Engineering
# =====================================================
def add_technical_indicators(df):
    df["RSI_14"] = ta.rsi(df["Close"], length=14)
    df["MACD"], df["MACD_SIGNAL"], df["MACD_HIST"] = ta.macd(df["Close"], fast=12, slow=26, signal=9).values.T
    df["SMA_20"] = ta.sma(df["Close"], length=20)
    df["EMA_20"] = ta.ema(df["Close"], length=20)
    df["Returns"] = df["Close"].pct_change()
    df.dropna(inplace=True)
    return df

processed = {}

for ticker, df in all_data.items():
    df = add_technical_indicators(df)
    df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)  # 1 if next day up, else 0
    processed[ticker] = df
    df.to_csv(f"data/processed/{ticker}_processed.csv", index=False)
    print(f"✅ Processed {ticker} data: {df.shape[0]} rows, {df.shape[1]} columns")

# =====================================================
# 3️⃣ Merge for Multi-stock Dataset (Optional)
# =====================================================
combined = pd.concat(processed.values(), keys=processed.keys(), names=["Ticker", "Row"])
combined.to_csv("data/processed/all_stocks_combined.csv")
print(f"✅ Combined dataset shape: {combined.shape}")


/tmp/ipython-input-3521894214.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


NameError: name 'all_data' is not defined